In [ ]:
%reload_ext autoreload
%autoreload 2

import os
from tqdm import tqdm
import json
import datetime as dt
import numpy as np
import pandas as pd

In [ ]:
os.chdir("../src")

from gpg.data_retrieval.config import *
from gpg.data_retrieval.client import EntsoeClient

In [ ]:
with open("../entsoe-api-key.txt", "r") as f:
    api_token = json.load(f)["apiToken"]

In [ ]:
# Define client
client = EntsoeClient(api_token=api_token)

# Define start and end dates for query
start_date = dt.datetime(2015, 1, 1, 0, 0, 0)
end_date = dt.datetime(2021, 8, 1, 0, 0, 0)
date_range = pd.date_range(start_date, end_date, freq="MS")

# Run data retrieval
dfs = []
with tqdm(list(zip(date_range, date_range[1:])), desc="Data retrieval for", total=len(date_range)-1) as t:
    for (start, end) in t:
        t.set_description(
            f"Data retrieval for {start.strftime('%Y-%m-%d')} - {end.strftime('%Y-%m-%d')}"
        )
        dfs.append(
            client.extract_data(
                client.query(
                    document=Document.total_load,
                    process=Process.actual,
                    area=Area.hertz_50,
                    dt_start=start,
                    dt_end=end
                ),
                convert_utc_to_local_tz=False
            )
        )

# Concat data
df = pd.concat(dfs).set_index("timestamp")

# To csv
df.to_csv("../data/de-50hertz-demand-data.csv")